In [1]:
!pip install fuzzywuzzy
!pip install python-Levenshtein
import pandas as pd
import numpy as np
#import sqlite3
from fuzzywuzzy import fuzz

You should consider upgrading via the 'pip install --upgrade pip' command.
     |████████████████████████████████| 51kB 533kB/s eta 0:00:01
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp35-cp35m-linux_x86_64.whl size=152477 sha256=0ecc467c663d63d8ac2a718b718b746bf41f17a260d0dfb64260331659cd9975
  Stored in directory: /home/nbuser/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
#read necessary tables from sqlite database to pd.Dataframes
'''con = sqlite3.connect('european_soccer_data.sqlite')
match_table = pd.read_sql_query('SELECT * from Match', con)
team_table = pd.read_sql_query('SELECT * from Team', con)
country_table = pd.read_sql_query('SELECT * from Country', con)
con.close()'''

#write to csv to make future edits easier to start
'''match_csv = match_table.to_csv('match.csv', index=False)
team_csv = team_table.to_csv('team.csv', index=False)
country_csv = country_table.to_csv('country.csv', index=False)'''

#read csv to pd.Dataframe
#match_df = pd.read_csv('match.csv')
team_df = pd.read_csv('team.csv')
country_df = pd.read_csv('country.csv')

In [ ]:
country_df.head()

In [ ]:
team_df.head()

In [ ]:
match_df.head()

In [ ]:
#manual league name dict for leagues of interest, small dataset so manual works
league_name_dict = {'England':'Premier League', 'France':'Ligue 1', 'Germany':'1 Bundesliga', 'Italy':'Serie A',
                    'Netherlands':'Eredivisie', 'Portugal':'Liga Nos', 'Spain':'Primera Division'}

#add league name column to country_df
country_df['league_name'] = ''
for i, row in country_df.iterrows():
    league_name = 'NA'
    if row['name'] in league_name_dict.keys():
        league_name = league_name_dict[row['name']]
    country_df.at[i, 'league_name'] = league_name

#subset country_df to leagues I have match data for
country_df = country_df[country_df.league_name != 'NA'].reset_index(drop=True)
country_df

In [ ]:
#create dict of {team id: team name} and {country id: league name} which will be used to create new columns in match_df
team_id_dict = pd.Series(team_df.team_long_name.values, index=team_df.team_api_id).to_dict()
country_id_dict = pd.Series(country_df.league_name.values, index=country_df.id).to_dict()

In [ ]:
#add empty column to specific index in match_df, can't run this code multiple times, comment out
match_df.insert(7, 'home_team_name', ['' for i in range(match_df.shape[0])])
match_df.insert(8, 'away_team_name', ['' for i in range(match_df.shape[0])])
match_df.insert(3, 'league_name', ['' for i in range(match_df.shape[0])])

for i, row in match_df.iterrows():
    league_name = 'NA'
    if row['country_id'] in country_id_dict.keys():
        league_name = country_id_dict[row['country_id']]
    match_df.at[i, 'league_name'] = league_name

#subset match_df to leagues I have transfer data for
match_df = match_df[match_df.league_name != 'NA'].reset_index(drop=True)

for i, row in match_df.iterrows():
    match_df.at[i, 'home_team_name'] = team_id_dict[row['home_team_api_id']]
    match_df.at[i, 'away_team_name'] = team_id_dict[row['away_team_api_id']]

match_df.head()

In [ ]:
#removing some unnecessary columns, can only run once, comment out
for i in range(1,12):
    match_df = match_df.drop(['home_player_X'+str(i),'away_player_X'+str(i),'home_player_Y'+str(i),
                              'away_player_Y'+str(i),'home_player_'+str(i),'away_player_'+str(i)], axis=1)

for column in match_df.columns[-30:]:
    match_df = match_df.drop(column, axis=1)

match_df = match_df.drop(['country_id', 'id'], axis=1)

print(match_df.shape)
print([column for column in match_df.columns])
match_df.head()

In [ ]:
#edit season column to int of start year rather than string of years, ex. 2008 refers to 2008/2009 season
match_df.season = match_df.season.apply(lambda x : int(x[:4]))
match_df.head()

In [ ]:
#add home team win, loss, draw columns
match_df.insert(10, 'home_win', [False for i in range(match_df.shape[0])])
match_df.insert(11, 'away_win', [False for i in range(match_df.shape[0])])
match_df.insert(12, 'draw', [False for i in range(match_df.shape[0])])

In [ ]:
#loop through rows, update win, loss, draw columns to True where necessary
for i, row in match_df.iterrows():
    if row['home_team_goal'] > row['away_team_goal']:
        match_df.at[i, 'home_win'] = True
    elif row['home_team_goal'] < row['away_team_goal']:
        match_df.at[i, 'away_win'] = True
    else:
        match_df.at[i, 'draw'] = True

match_df[['home_team_goal', 'away_team_goal', 'home_win', 'away_win', 'draw']].head()

In [ ]:
#sort match_df by league, season, stage
match_df = match_df.groupby(['season', 'league_name']).apply(lambda x: x.sort_values(['stage'])).reset_index(drop=True)
match_df.head(50)

In [3]:
#match_clean_csv = match_df.to_csv('match_clean.csv')
match_df_clean = pd.read_csv('match_clean.csv')

In [4]:
def get_league_table(season, league_name): # returns dataframe of end of season table for given year, league
    season_df = match_df_clean[(match_df_clean['league_name']==league_name) & (match_df_clean['season']==season)] #subset match_df to specified season, league
    
    league_table_df = pd.DataFrame(0, columns=['Wins', 'Draws', 'Losses', 'Goals_For', 'Goals_Against', 'Goal_Difference', 'Points'],
                                   index=season_df['home_team_name'].unique()).rename_axis('Team') #initialize every column to zero, team names as indices
        
    for i, row in season_df.iterrows(): #update goals for/against, win/loss/draw columns
        league_table_df.loc[row['home_team_name'], 'Goals_For'] += row['home_team_goal']
        league_table_df.loc[row['home_team_name'], 'Goals_Against'] += row['away_team_goal']
        league_table_df.loc[row['away_team_name'], 'Goals_For'] += row['away_team_goal']
        league_table_df.loc[row['away_team_name'], 'Goals_Against'] += row['home_team_goal']
        
        if row['home_win'] == True:
            league_table_df.loc[row['home_team_name'], 'Wins'] += 1
            league_table_df.loc[row['away_team_name'], 'Losses'] += 1
        elif row['away_win'] == True:
            league_table_df.loc[row['home_team_name'], 'Losses'] += 1
            league_table_df.loc[row['away_team_name'], 'Wins'] += 1
        else:
            league_table_df.loc[row['home_team_name'], 'Draws'] += 1
            league_table_df.loc[row['away_team_name'], 'Draws'] += 1
        
    league_table_df['Goal_Difference'] = league_table_df['Goals_For'] - league_table_df['Goals_Against']
    league_table_df['Points'] = league_table_df['Wins']*3 + league_table_df['Draws']
    league_table_df = league_table_df.sort_values(['Points', 'Goal_Difference'], ascending=False).reset_index()
    league_table_df.index = pd.RangeIndex(start=1, stop=len(league_table_df)+1, step=1)
    
    return league_table_df

get_league_table(2008, 'Premier League')

,Team,Wins,Draws,Losses,Goals_For,Goals_Against,Goal_Difference,Points
1,Manchester United,28,6,4,68,24,44,90
2,Liverpool FC,25,11,2,77,27,50,86
3,Chelsea FC,25,8,5,68,24,44,83
4,Arsenal FC,20,12,6,68,37,31,72
5,Everton FC,17,12,9,55,37,18,63
6,Aston Villa,17,11,10,54,48,6,62
7,Fulham FC,14,11,13,39,34,5,53
8,Tottenham Hotspur,14,9,15,45,45,0,51
9,West Ham United,14,9,15,42,45,-3,51
10,Manchester City,15,5,18,58,50,8,50


In [5]:
#returns league finish of given team at end of given season
def get_league_finish(season, team_name):
    league = match_df_clean.league_name[match_df_clean.home_team_name==team_name].tolist()[0]
    league_table = get_league_table(season, league)
    finish = league_table[league_table['Team'] == team_name].index.values.astype(int)[0]
    return finish

get_league_finish(2008, 'Arsenal FC')

4

In [6]:
filenames = ['dutch_eredivisie.csv', 'english_premier_league.csv', 'french_ligue_1.csv', 'german_bundesliga_1.csv',
                'italian_serie_a.csv', 'portugese_liga_nos.csv', 'spanish_primera_division.csv']

transfers_df_list = []

for i in range(2007, 2017):
    for file in filenames:
        df = pd.read_csv('transfer_data/'+str(i)+'/'+file)
        df.insert(0, 'club_api_id', '')
        transfers_df_list.append(df)

transfers_df = pd.concat(transfers_df_list).reset_index(drop=True)

transfers_df.head()

,club_api_id,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,fee_cleaned,league_name,year,season
0,,PSV Eindhoven,Danny Koevermans,28.0,Centre-Forward,AZ Alkmaar,£6.21m,in,6.21,Eredivisie,2007,2007/2008
1,,PSV Eindhoven,Danko Lazovic,24.0,Second Striker,Vitesse,£5.40m,in,5.40,Eredivisie,2007,2007/2008
2,,PSV Eindhoven,Balázs Dzsudzsák,21.0,Left Winger,Debrecen,£2.25m,in,2.25,Eredivisie,2007,2007/2008
3,,PSV Eindhoven,Cássio,20.0,Goalkeeper,Grêmio,£900k,in,0.90,Eredivisie,2007,2007/2008
4,,PSV Eindhoven,Kenneth Pérez,32.0,Centre-Forward,Ajax,£900k,in,0.90,Eredivisie,2007,2007/2008


In [15]:
transfers_df.rename(columns={'fee_cleaned':'fee_cleaned_millions'}, inplace=True)
transfers_in = transfers_df[transfers_df['transfer_movement'] == 'in'] #subset purchases
transfers_out = transfers_df[transfers_df['transfer_movement'] == 'out'] #subset sales

transfers_in.head()

,club_api_id,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,fee_cleaned_millions,league_name,year,season
0,,PSV Eindhoven,Danny Koevermans,28.0,Centre-Forward,AZ Alkmaar,£6.21m,in,6.21,Eredivisie,2007,2007/2008
1,,PSV Eindhoven,Danko Lazovic,24.0,Second Striker,Vitesse,£5.40m,in,5.40,Eredivisie,2007,2007/2008
2,,PSV Eindhoven,Balázs Dzsudzsák,21.0,Left Winger,Debrecen,£2.25m,in,2.25,Eredivisie,2007,2007/2008
3,,PSV Eindhoven,Cássio,20.0,Goalkeeper,Grêmio,£900k,in,0.90,Eredivisie,2007,2007/2008
4,,PSV Eindhoven,Kenneth Pérez,32.0,Centre-Forward,Ajax,£900k,in,0.90,Eredivisie,2007,2007/2008


In [16]:
forward_list = ['Centre-Forward', 'Second Striker', 'Left Winger', 'Right Winger', 'Forward']
midfield_list = ['Defensive Midfield', 'Attacking Midfield', 'Right Midfield', 'Left Midfield', 'Central Midfield', 'Midfielder']
defender_list = ['Goalkeeper', 'Left-Back', 'Centre-Back', 'Right-Back', 'Defender']

In [17]:
transfers_in.loc[transfers_in.position.isin(forward_list), 'position'] = 'Forward'
transfers_in.loc[transfers_in.position.isin(midfield_list), 'position'] = 'Midfielder'
transfers_in.loc[transfers_in.position.isin(defender_list), 'position'] = 'Defender'

transfers_out.loc[transfers_out.position.isin(forward_list), 'position'] = 'Forward'
transfers_out.loc[transfers_out.position.isin(midfield_list), 'position'] = 'Midfielder'
transfers_out.loc[transfers_out.position.isin(defender_list), 'position'] = 'Defender'

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [19]:
#money spent per club per year, avg age players bought per year
transfers_in_by_year = transfers_in.groupby(['club_name', 'club_api_id', 'year']).agg({'fee_cleaned_millions':'sum', 'age':'mean'})
transfers_in_by_year = transfers_in_by_year.rename(columns={'fee_cleaned_millions':'total_spent_millions', 'age':'average_age'})
transfers_in_by_year = transfers_in_by_year.reset_index(level=['club_api_id','year'])

transfers_out_by_year = transfers_out.groupby(['club_name', 'club_api_id', 'year']).agg({'fee_cleaned_millions':'sum', 'age':'mean'})
transfers_out_by_year = transfers_out_by_year.rename(columns={'fee_cleaned_millions':'total_sales_millions', 'age':'average_age'})
transfers_out_by_year = transfers_out_by_year.reset_index(level=['club_api_id','year'])

transfers_out_by_year.head()

,club_api_id,year,total_sales_millions,average_age
club_name,,,,
1. FC Köln,,2008,0.675,25.769231
1. FC Köln,,2009,0.810,26.666667
1. FC Köln,,2010,0.045,25.000000
1. FC Köln,,2011,2.585,23.285714
1. FC Köln,,2014,0.090,23.533333


In [20]:
in_df_list = []
for name, df in transfers_in_by_year.groupby(level='club_name'):
    df.insert(0, 'team_name_test', '')
    if name in list(team_df.team_long_name):
        df.loc[name, 'club_api_id'] = team_df.set_index('team_long_name').loc[name,'team_api_id']
        df.loc[name, 'team_name_test'] = name
    else:
        for team in list(team_df.team_long_name):
            if fuzz.partial_ratio(name, team) >= 77:
                df.loc[name, 'club_api_id'] = team_df.set_index('team_long_name').loc[team,'team_api_id']
                df.loc[name, 'team_name_test'] = team
                break
    in_df_list.append(df)

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [21]:
#verify all clubs without an id do not have match data
transfers_in_by_year = pd.concat(in_df_list)
transfers_in_by_year[transfers_in_by_year.club_api_id=='']

,team_name_test,club_api_id,year,total_spent_millions,average_age
club_name,,,,,
CD Leganés,,,2016,3.195,25.318182
Deportivo Alavés,,,2016,5.695,24.105263
Derby County,,,2007,20.413,26.000000
FC Crotone,,,2016,5.310,22.729730
FC Hansa Rostock,,,2007,0.630,23.916667
GD Chaves,,,2016,0.045,24.906250
MSV Duisburg,,,2007,3.600,25.722222
RB Leipzig,,,2016,70.610,21.583333
RC Strasbourg Alsace,,,2007,1.215,23.227273


In [22]:
#manually check to ensure code worked for all imperfectly matched names
for df in in_df_list:
    if df.index[0] != df['team_name_test'].tolist()[0]:
        print('Club Name: ', df.index[0]+'\n', 'Test Name:', df['team_name_test'].tolist()[0]+'\n')

Club Name:  1.FC Kaiserslautern
 Test Name: 1. FC Kaiserslautern

Club Name:  1.FC Nuremberg
 Test Name: 1. FC Nürnberg

Club Name:  AC Cesena
 Test Name: Cesena

Club Name:  AS Livorno
 Test Name: Livorno

Club Name:  Académica Coimbra
 Test Name: Académica de Coimbra

Club Name:  Arminia Bielefeld
 Test Name: DSC Arminia Bielefeld

Club Name:  Belenenses SAD
 Test Name: CF Os Belenenses

Club Name:  Blackpool FC
 Test Name: Blackpool

Club Name:  Brescia Calcio
 Test Name: Brescia

Club Name:  CD Feirense
 Test Name: Feirense

Club Name:  CD Leganés
 Test Name: 

Club Name:  CD Tondela
 Test Name: Tondela

Club Name:  CD Trofense
 Test Name: Trofense

Club Name:  CF Estrela Amadora
 Test Name: Amadora

Club Name:  Celta de Vigo
 Test Name: RC Celta de Vigo

Club Name:  De Graafschap Doetinchem
 Test Name: De Graafschap

Club Name:  Delfino Pescara 1936
 Test Name: Pescara

Club Name:  Deportivo Alavés
 Test Name: 

Club Name:  Deportivo de La Coruña
 Test Name: RC Deportivo de La Cor

In [25]:
#code worked for transfers_in_by_year, do the same for transfers_out_by_year
out_df_list = []
for name, df in transfers_out_by_year.groupby(level='club_name'):
    df.insert(0, 'team_name_test', '')
    if name in list(team_df.team_long_name):
        df.loc[name, 'club_api_id'] = team_df.set_index('team_long_name').loc[name,'team_api_id']
        df.loc[name, 'team_name_test'] = name
    else:
        for team in list(team_df.team_long_name):
            if fuzz.partial_ratio(name, team) >= 77:
                df.loc[name, 'club_api_id'] = team_df.set_index('team_long_name').loc[team,'team_api_id']
                df.loc[name, 'team_name_test'] = team
                break
    out_df_list.append(df)

transfers_out_by_year = pd.concat(out_df_list)

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [28]:
#total spent/sold by club between 2007-2016
total_spent = transfers_in_by_year.groupby(level=0).agg({'average_age':'mean', 'total_spent_millions':'sum'}).sort_values('total_spent_millions', ascending=False)
total_sold = transfers_out_by_year.groupby(level=0).agg({'average_age':'mean', 'total_sales_millions':'sum'}).sort_values('total_sales_millions', ascending=False)

#top 100 spending clubs during period of interest
top_100_spenders = total_spent.head(100)

top_100_spenders

,total_spent_millions,average_age
club_name,,
Manchester City,1210.796,23.008789
Real Madrid,955.530,23.400574
Manchester United,861.925,21.367788
Chelsea FC,844.670,21.630092
FC Barcelona,798.460,23.328096
Liverpool FC,769.633,22.010123
Juventus FC,766.091,22.538188
Paris Saint-Germain,674.640,23.554886
Tottenham Hotspur,640.060,22.069570
